# Assignment

In this assignment we will train a box localization algorithm derived from RetinaNet to perform identification of brain tumors on MRI. The algorithm will be implemented using a feature pyramid network backbone. Accuracy will be calculated based on median IoU performance against ground-truth masks.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook (push to https://github.com/[username]/cs190/cnn/assignment.ipynb)
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Tensorflow 2.1

This tutorial specifically requires the use of Tensorflow 2.1 for implemention of custom (weighted) loss functions. Use the following command to install this specific library version:

In [1]:
% pip install tensorflow-gpu==2.1

     |████████████████████████████████| 421.8MB 19kB/s 
     |████████████████████████████████| 450kB 46.5MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 3.9MB 36.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=02993a8df4576df7b819a7b3efacee54e86e3dfcde58797c9a44c45b31a0dc8f
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installa

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [2]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 81kB 4.3MB/s 
     |████████████████████████████████| 645kB 8.8MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [3]:
import numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers, metrics
from jarvis.train import datasets, custom
from jarvis.train.box import BoundingBox

# Data

The data used in this assignment will consist of brain tumor MRI exams derived from the MICCAI Brain Tumor Segmentation Challenge (BRaTS). More information about he BRaTS Challenge can be found here: http://braintumorsegmentation.org/. Each single 2D slice will consist of one of four different sequences (T2, FLAIR, T1 pre-contrast and T1 post-contrast).

The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset

In [4]:
# --- Download dataset
datasets.download(name='mr/brats-2020-mip')

# --- Prepare generators and model inputs
configs = {'specs': {'ys': {'tumor': {'norms': {'clip': {'max': 1}}}}}}
gen_train, gen_valid, client = datasets.prepare(name='mr/brats-2020-mip', keyword='mip*vox', configs=configs)

[ 2021-05-18 03:48:04 ] [====================] 100.000% : Extracting archive (0000750 / 0000750) 

# Training

In this assignment we will train a box localization network for brain tumor segmentation.

### Define box parameters

Use the following cell block to define your `BoundingBox` object as discussed in the tutorial. Feel free to optimize hyperparameter choices for grid size, anchor shapes, anchor aspect ratios, and anchor scales: 

In [5]:
bb = BoundingBox(
    image_shape=(240, 240),
    classes=1,
    c=[3, 4],
    anchor_shapes=[32, 64],
    anchor_scales=[0, 1, 2],
    anchor_ratios=[0.5, 1, 2],
    iou_upper=0.5,
    iou_lower=0.2)

### Define inputs

Use the following cell block to define the nested generators needed to convert raw masks into bounding box ground-truth predictions:

In [7]:
# --- Prepare generators
gen_train, gen_valid = client.create_generators()
gen_train, gen_valid = bb.create_generators(gen_train, gen_valid, msk='tumor')

# --- Create inputs
inputs = client.get_inputs(Input)
inputs = bb.get_inputs(inputs, Input)

### Define the model

Use the following cell block to define your feature pyramid network backbone and RetinaNet classification / regression networks:

In [8]:

# --- Define kwargs dictionary
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'}

# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))

# --- Define contracting layers
l1 = conv1(8, inputs['dat'])
l2 = conv1(16, conv2(16, l1))
l3 = conv1(24, conv2(24, l2))
l4 = conv1(32, conv2(32, l3))
l5 = conv1(48, conv2(48, l4))

# --- Define zoom
zoom = lambda x : layers.UpSampling3D(
    size=(1, 2, 2))(x)


# --- Define 1 x 1 x 1 projection
proj = lambda filters, x : layers.Conv3D(
    filters=filters,
    strides=1,
    kernel_size=(1, 1, 1),
    padding='same',
    kernel_initializer='he_normal')(x)

# --- Define expanding layers
l6 = proj(64, l5)
l7 = conv1(64, zoom(l6) + proj(64, l4))

# --- Determine filter sizes
logits = {}
K = 1
A = 9

# --- C3
c3_cls = conv1(64, conv1(64, l7))
c3_reg = conv1(64, conv1(64, l7))
logits['cls-c3'] = layers.Conv3D(filters=(A * K), name='cls-c3', **kwargs)(c3_cls)
logits['reg-c3'] = layers.Conv3D(filters=(A * 4), name='reg-c3', **kwargs)(c3_reg)

# --- C4
c4_cls = conv1(64, conv1(64, l6))
c4_reg = conv1(64, conv1(64, l6))
logits['cls-c4'] = layers.Conv3D(filters=(A * K), name='cls-c4', **kwargs)(c4_cls)
logits['reg-c4'] = layers.Conv3D(filters=(A * 4), name='reg-c4', **kwargs)(c4_reg)

model = Model(inputs=inputs, outputs=logits)


### Compile the model

Use the following cell block to compile your model. Recall the following requirements as described in the tutorial:

* use of a focal sigmoid (binary) cross-entropy loss function for regression
* use of a Huber loss function for classification
* use of masked loss functions to ensure only relevant examples are used for training
* use of appropriate metrics to track algorithm training

In [9]:
def focal_sigmoid_ce(weights=1.0, scale=1.0, gamma=2.0, alpha=0.25):
    """
    Method to implement focal sigmoid (binary) cross-entropy loss

    """
    def focal_sigmoid_ce(y_true, y_pred):

        # --- Calculate standard cross entropy with alpha weighting
        loss = tf.nn.weighted_cross_entropy_with_logits(
            labels=y_true, logits=y_pred, pos_weight=alpha)

        # --- Calculate modulation to pos and neg labels 
        p = tf.math.sigmoid(y_pred)
        modulation_pos = (1 - p) ** gamma
        modulation_neg = p ** gamma

        mask = tf.dtypes.cast(y_true, dtype=tf.bool)
        modulation = tf.where(mask, modulation_pos, modulation_neg)

        return tf.math.reduce_sum(modulation * loss * weights * scale)

    return focal_sigmoid_ce

In [10]:
# --- Create sensivity and PPV metrics
custom.sigmoid_ce_sens()
custom.sigmoid_ce_ppv()

<function jarvis.train.custom.custom.sigmoid_ce_ppv.<locals>.sigmoid_ce_ppv>

In [11]:
# --- Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={
        'cls-c3': custom.focal_sigmoid_ce(inputs['cls-c3-msk']),
        'cls-c4': custom.focal_sigmoid_ce(inputs['cls-c4-msk']),
        'reg-c3': custom.sl1(inputs['reg-c3-msk']),
        'reg-c4': custom.sl1(inputs['reg-c4-msk']),
        },
    metrics={
        'cls-c3': [custom.sigmoid_ce_sens(), custom.sigmoid_ce_ppv()],
        'cls-c4': [custom.sigmoid_ce_sens(), custom.sigmoid_ce_ppv()]},
    experimental_run_tf_function=False)

### Train the model

Use the following cell block to train your model.

In [12]:
client.load_data_in_memory()

[ 2021-05-18 03:58:08 ] [====================] 100.000% : Iterating | 000368    

In [13]:
# --- Train model
model.fit(
    x=gen_train, 
    steps_per_epoch=500, 
    epochs=16,
    validation_data=gen_valid,
    validation_steps=500,
    validation_freq=4,
    use_multiprocessing=True)

Epoch 1/16
500/500 [==============================] - 106s 212ms/step - loss: 0.0189 - cls-c3_loss: 0.0080 - cls-c4_loss: 0.0107 - reg-c3_loss: 9.3513e-05 - reg-c4_loss: 7.0688e-05 - cls-c3_sigmoid_ce_sens: 0.0552 - cls-c3_sigmoid_ce_ppv: 0.2486 - cls-c4_sigmoid_ce_sens: 0.0610 - cls-c4_sigmoid_ce_ppv: 0.2073
Epoch 2/16
500/500 [==============================] - 95s 189ms/step - loss: 8.9238e-04 - cls-c3_loss: 3.9759e-04 - cls-c4_loss: 4.2626e-04 - reg-c3_loss: 5.0901e-05 - reg-c4_loss: 1.7633e-05 - cls-c3_sigmoid_ce_sens: 0.2473 - cls-c3_sigmoid_ce_ppv: 0.4530 - cls-c4_sigmoid_ce_sens: 0.3873 - cls-c4_sigmoid_ce_ppv: 0.4803
Epoch 3/16
500/500 [==============================] - 94s 189ms/step - loss: 6.3086e-04 - cls-c3_loss: 2.8982e-04 - cls-c4_loss: 2.7362e-04 - reg-c3_loss: 5.2839e-05 - reg-c4_loss: 1.4581e-05 - cls-c3_sigmoid_ce_sens: 0.3646 - cls-c3_sigmoid_ce_ppv: 0.3925 - cls-c4_sigmoid_ce_sens: 0.5295 - cls-c4_sigmoid_ce_ppv: 0.4388
Epoch 4/16
499/500 [=========================

# Evaluation

Based on the tutorial discussion, use the following cells to calculate model performance. The following metrics should be calculated:

* median IoU
* 25th percentile IoU
* 75th percentile IoU

### Performance

The following minimum performance metrics must be met for full credit:

* median IoU: >0.50
* 25th percentile IoU: >0.40
* 75th percentile IoU: >0.60

In [16]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train, test_valid = bb.create_generators(test_train, test_valid, msk='tumor')

ious = {
    'med': [],
    'p25': [],
    'p75': []}

for x, y in test_valid:
    
    # --- Predict
    box = model.predict(x)
    if type(box) is list:
        box = {k: l for k, l in zip(model.output_names, box)}
        
    # --- Convert predictions to anchors
    anchors_pred, _ = bb.convert_box_to_anc(box, iou_nms=0.5)
    
    # --- Convert ground-truth to anchors
    anchors_true, _ = bb.convert_box_to_anc(y)
    
    # --- Calculate IoUs
    curr = []
    for pred, true in zip(anchors_pred, anchors_true):
        for p in pred:
            iou = bb.calculate_ious(box=p, anchors=true)
            if iou.size > 0:
                curr.append(np.max(iou))
            else: 
                curr.append(0)
    
    if len(curr) == 0:
        curr = [0]
        
    ious['med'].append(np.median(curr))
    ious['p25'].append(np.percentile(curr, 25))
    ious['p75'].append(np.percentile(curr, 75))
    
ious = {k: np.array(v) for k, v in ious.items()}

[ 2021-05-18 04:39:16 ] [====================] 100.000% : Iterating | 000074    

In [17]:
# --- Define columns
df = pd.DataFrame(index=np.arange(ious['med'].size))
df['iou_median'] = ious['med']
df['iou_p-25th'] = ious['p25']
df['iou_p-75th'] = ious['p75']

df.to_csv('./results.csv')
# --- Print accuracy
print(df['iou_median'].median())
print(df['iou_p-25th'].median())
print(df['iou_p-75th'].median())

0.5794649124145508
0.44157037883996964
0.7426700443029404


### Results

When ready, create a `*.csv` file with your compiled **validation** cohort IoU statistics. There is no need to submit training performance accuracy.

# Submission

Use the following line to save your model for submission (in Google Colab this should save your model file into your personal Google Drive):

In [18]:
# --- Serialize a model
model.save('./box_localization.hdf5')

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.